In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import io

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
print("Dataset Loaded Successfully!")
display(df.head())

Saving ProfileIdentification_Data.csv to ProfileIdentification_Data.csv
Dataset Loaded Successfully!


Age  Gender  Weight_kg  Height_m  Session_Duration_hours  \
0  34.91    Male      65.27      1.62                    1.00   
1  23.37  Female      56.41      1.55                    1.37   
2  33.20  Female      58.98      1.67                    0.91   
3  38.69  Female      93.78      1.70                    1.10   
4  45.09    Male      52.42      1.88                    1.08   

   Workout_Frequency_days_per_week    BMI Trainee_Profile  
0                             3.99  24.87    Intermediate  
1                             4.00  23.48    Intermediate  
2                             2.99  21.15        Beginner  
3                             3.99  32.45    Intermediate  
4                             4.00  14.83    Intermediate

In [ ]:
df["Height_cm"] = df["Height_m"] * 100
df["Session_Duration_minutes"] = df["Session_Duration_hours"] * 60
df.drop(columns=["Height_m", "Session_Duration_hours"], inplace=True)


In [ ]:
num_cols = [
    "Age", "Weight_kg", "Height_cm",
    "Session_Duration_minutes",
    "Workout_Frequency_days_per_week",
    "BMI"
]
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

df["Gender"] = df["Gender"].fillna(df["Gender"].mode()[0])
df["Trainee_Profile"] = df["Trainee_Profile"].fillna(df["Trainee_Profile"].mode()[0])


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df["Workout_Frequency_days_per_week"] = np.floor(df["Workout_Frequency_days_per_week"]).astype(int)
df["Workout_Frequency_days_per_week"] = df["Workout_Frequency_days_per_week"].clip(1, 7)  # keep within 1-7

In [ ]:
def assign_workout_plan(profile, bmi):
    if profile == "Beginner":
        if bmi < 18.5:
            return "Beginner 1"
        elif 18.5 <= bmi <= 22.9:
            return "Beginner 2"
        elif 23 <= bmi <= 24.9:
            return "Beginner 3"
        elif 25 <= bmi <= 29.9:
            return "Beginner 4"
        else:
            return "Beginner 5"

    elif profile == "Intermediate":
        if bmi < 18.5:
            return "Intermediate 1"
        elif 18.5 <= bmi <= 22.9:
            return "Intermediate 2"
        elif 23 <= bmi <= 24.9:
            return "Intermediate 3"
        elif 25 <= bmi <= 29.9:
            return "Intermediate 4"
        else:
            return "Intermediate 5"

    elif profile == "Advanced":
        if bmi < 18.5:
            return "Advanced 1"
        elif 18.5 <= bmi <= 22.9:
            return "Advanced 2"
        elif 23 <= bmi <= 24.9:
            return "Advanced 3"
        elif 25 <= bmi <= 29.9:
            return "Advanced 4"
        else:
            return "Advanced 5"

    # fallback if unexpected value
    return "Beginner 1"


df["Workout_Plan_Label"] = df.apply(
    lambda row: assign_workout_plan(row["Trainee_Profile"], row["BMI"]),
    axis=1
)

In [ ]:
df["Weight_Height_Ratio"] = df["Weight_kg"] / df["Height_cm"]
df["Duration_per_Week"] = df["Session_Duration_minutes"] * df["Workout_Frequency_days_per_week"]
df["BMI_Age_Ratio"] = df["BMI"] / df["Age"].replace(0, np.nan)
df["BMI_Age_Ratio"] = df["BMI_Age_Ratio"].fillna(df["BMI_Age_Ratio"].mean())

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_gender = LabelEncoder()
df["Gender_enc"] = le_gender.fit_transform(df["Gender"].astype(str).str.strip().str.capitalize())

le_workout = LabelEncoder()
df["Workout_Plan_enc"] = le_workout.fit_transform(df["Workout_Plan_Label"])

# ✅ IMPORTANT: define feature columns ONCE (used for train + predict)
feature_cols = [
    "Age", "Gender_enc", "Weight_kg", "Height_cm",
    "Session_Duration_minutes", "Workout_Frequency_days_per_week", "BMI",
    "Weight_Height_Ratio", "Duration_per_Week", "BMI_Age_Ratio"
]

X = df[feature_cols].copy()
y = df["Workout_Plan_enc"].copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Training samples:", X_train.shape)
print("Testing samples :", X_test.shape)

Training samples: (15999, 10)
Testing samples : (4000, 10)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=5,
    min_samples_split=5,
    max_features="sqrt",
    class_weight="balanced",
    random_state=42
)

model.fit(X_train, y_train)
print("\nModel trained!")


Model trained!


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

y_pred = model.predict(X_test)
print("\nTest Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred))

y_train_pred = model.predict(X_train)
print("Train Accuracy:", round(accuracy_score(y_train, y_train_pred), 3))

cv_scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
print("CV Mean:", round(cv_scores.mean(), 3))
print("CV Std :", round(cv_scores.std(), 3))


Test Accuracy: 0.959
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00       211
           2       1.00      1.00      1.00       203
           3       1.00      1.00      1.00       216
           4       0.99      0.99      0.99       118
           5       1.00      0.97      0.98       318
           6       0.98      0.92      0.95       384
           7       0.96      0.97      0.96       161
           8       0.90      0.89      0.90       342
           9       0.98      0.96      0.97       346
          10       0.97      1.00      0.98       329
          11       0.92      0.97      0.95       351
          12       0.93      0.98      0.96       193
          13       0.90      0.93      0.91       418
          14       0.99      0.94      0.96       378

    accuracy                           0.96      4000
   macro avg       0.97      0.97      0.97      4000
weig

In [ ]:
import joblib

joblib.dump(model, "workout_plan.pkl")
joblib.dump(le_gender, "gender_encoder.pkl")
joblib.dump(le_workout, "workout_plan_encoder.pkl")
joblib.dump(feature_cols, "feature_columns.pkl")

print("✅ Saved in Colab runtime")


✅ Saved in Colab runtime


In [ ]:
from google.colab import files

files.download("workout_plan.pkl")
files.download("gender_encoder.pkl")
files.download("workout_plan_encoder.pkl")
files.download("feature_columns.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# ==============================
# 1) UPLOAD ZIP (one file)
# ==============================
from google.colab import files
import zipfile
import os
import joblib
import pandas as pd

print("📌 Upload: SmartAR_Model_Package.zip")
uploaded = files.upload()

zip_name = list(uploaded.keys())[0]  # takes the uploaded zip file name

# ==============================
# 2) EXTRACT ZIP
# ==============================
extract_dir = "/content/SmartAR_Model_Package"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall("/content")

print("✅ Extracted files!")

# ==============================
# 3) LOAD MODEL FILES FROM FIXED PATH
# ==============================
models_dir = "/content/SmartAR_Model_Package/models"

required_files = [
    "workout_plan.pkl",
    "gender_encoder.pkl",
    "workout_plan_encoder.pkl",
    "feature_columns.pkl"
]

missing = [f for f in required_files if not os.path.exists(os.path.join(models_dir, f))]
if missing:
    raise FileNotFoundError(f"❌ Missing files in models folder: {missing}")

clf = joblib.load(os.path.join(models_dir, "workout_plan.pkl"))
le_gender = joblib.load(os.path.join(models_dir, "gender_encoder.pkl"))
le_workout = joblib.load(os.path.join(models_dir, "workout_plan_encoder.pkl"))
feature_cols = joblib.load(os.path.join(models_dir, "feature_columns.pkl"))

print("✅ Loaded model + encoders!")

# ==============================
# WORKOUT PLAN TABLE
# ==============================
WORKOUT_PLANS = {
    "Beginner 1":  {"Jumping Jacks": 10, "Push-ups": 1, "Squats": 3, "Crunches": 5, "Plank(sec)": 10, "Cobra(sec)": 15},
    "Beginner 2":  {"Jumping Jacks": 15, "Push-ups": 2, "Squats": 5, "Crunches": 7, "Plank(sec)": 15, "Cobra(sec)": 15},
    "Beginner 3":  {"Jumping Jacks": 20, "Push-ups": 2, "Squats": 8, "Crunches": 10, "Plank(sec)": 20, "Cobra(sec)": 20},
    "Beginner 4":  {"Jumping Jacks": 12, "Push-ups": 1, "Squats": 5, "Crunches": 6, "Plank(sec)": 15, "Cobra(sec)": 20},
    "Beginner 5":  {"Jumping Jacks": 10, "Push-ups": 1, "Squats": 5, "Crunches": 5, "Plank(sec)": 10, "Cobra(sec)": 20},

    "Intermediate 1": {"Jumping Jacks": 20, "Push-ups": 3, "Squats": 12, "Crunches": 12, "Plank(sec)": 25, "Cobra(sec)": 25},
    "Intermediate 2": {"Jumping Jacks": 25, "Push-ups": 4, "Squats": 15, "Crunches": 15, "Plank(sec)": 30, "Cobra(sec)": 30},
    "Intermediate 3": {"Jumping Jacks": 30, "Push-ups": 5, "Squats": 18, "Crunches": 18, "Plank(sec)": 35, "Cobra(sec)": 35},
    "Intermediate 4": {"Jumping Jacks": 20, "Push-ups": 4, "Squats": 14, "Crunches": 14, "Plank(sec)": 25, "Cobra(sec)": 30},
    "Intermediate 5": {"Jumping Jacks": 20, "Push-ups": 3, "Squats": 12, "Crunches": 12, "Plank(sec)": 20, "Cobra(sec)": 30},

    "Advanced 1": {"Jumping Jacks": 40, "Push-ups": 7, "Squats": 22, "Crunches": 22, "Plank(sec)": 45, "Cobra(sec)": 45},
    "Advanced 2": {"Jumping Jacks": 45, "Push-ups": 8, "Squats": 25, "Crunches": 25, "Plank(sec)": 50, "Cobra(sec)": 50},
    "Advanced 3": {"Jumping Jacks": 50, "Push-ups": 10, "Squats": 30, "Crunches": 30, "Plank(sec)": 60, "Cobra(sec)": 60},
    "Advanced 4": {"Jumping Jacks": 40, "Push-ups": 8, "Squats": 25, "Crunches": 25, "Plank(sec)": 50, "Cobra(sec)": 50},
    "Advanced 5": {"Jumping Jacks": 40, "Push-ups": 6, "Squats": 20, "Crunches": 20, "Plank(sec)": 45, "Cobra(sec)": 45},
}

📌 Upload: SmartAR_Model_Package.zip


Saving SmartAR_Model_Package.zip to SmartAR_Model_Package (1).zip
✅ Extracted files!
✅ Loaded model + encoders!


In [4]:


# ==============================
# USER INPUT
# ==============================
age = int(input("Enter Age (years): "))
gender = input("Enter Gender (Male/Female): ").strip().capitalize()
weight_kg = float(input("Enter Weight (kg): "))
height_cm = float(input("Enter Height (cm): "))
session_duration_min = float(input("Enter session duration per day (minutes): "))
workout_days = int(input("Enter workout days per week (1–7): "))

workout_days = max(1, min(7, workout_days))

# Encode gender
try:
    gender_enc = le_gender.transform([gender])[0]
except ValueError:
    print(f"\n❌ Gender '{gender}' not found in training data.")
    print("✅ Allowed genders:", list(le_gender.classes_))
    raise

# ==============================
# FEATURE ENGINEERING
# ==============================
bmi = weight_kg / ((height_cm / 100) ** 2)
weight_height_ratio = weight_kg / height_cm
duration_per_week = session_duration_min * workout_days
bmi_age_ratio = bmi / age

X_input = pd.DataFrame([{
    "Age": age,
    "Gender_enc": gender_enc,
    "Weight_kg": weight_kg,
    "Height_cm": height_cm,
    "Session_Duration_minutes": session_duration_min,
    "Workout_Frequency_days_per_week": workout_days,
    "BMI": bmi,
    "Weight_Height_Ratio": weight_height_ratio,
    "Duration_per_Week": duration_per_week,
    "BMI_Age_Ratio": bmi_age_ratio
}])

# Ensure correct columns + order
X_input = X_input.reindex(columns=feature_cols)

# ==============================
# PREDICTION
# ==============================
pred = clf.predict(X_input)[0]
plan = le_workout.inverse_transform([pred])[0]

print("\n--- Prediction Result ---")
print(f"BMI: {bmi:.2f}")
print(f"Predicted Workout Plan: {plan}")

if plan in WORKOUT_PLANS:
    print("\n====== Workout Plan ======")
    for exercise, value in WORKOUT_PLANS[plan].items():
        print(f"{exercise}: {value}")
    print("============================")
else:
    print("⚠ Workout plan not found in plan dictionary.")


Enter Age (years): 25
Enter Gender (Male/Female): female
Enter Weight (kg): 53
Enter Height (cm): 149
Enter session duration per day (minutes): 0
Enter workout days per week (1–7): 0

--- Prediction Result ---
BMI: 23.87
Predicted Workout Plan: Beginner 3

====== Workout Plan ======
Jumping Jacks: 20
Push-ups: 2
Squats: 8
Crunches: 10
Plank(sec): 20
Cobra(sec): 20
